In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Demographic?? Based Recommender

In [352]:
import warnings; warnings.simplefilter('ignore')
md = pd.read_csv('C:/Users/xiaoj/Desktop/CS/collection of data science-take home challenges/kaggle-Recommendation/movies_metadata.csv')
md.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [182]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [183]:
from ast import literal_eval
md['genres'] = md['genres'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else 0)
md['genres']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres, Length: 45466, dtype: object

In [184]:
md[md['vote_average'].notnull()]['vote_average']

0        7.7
1        6.9
2        6.5
3        6.1
4        5.7
        ... 
45461    4.0
45462    9.0
45463    3.8
45464    0.0
45465    0.0
Name: vote_average, Length: 45460, dtype: float64

In [74]:
md[md['vote_average'].notnull()]['vote_average'].astype('int')

0        7
1        6
2        6
3        6
4        5
        ..
45461    4
45462    9
45463    3
45464    0
45465    0
Name: vote_average, Length: 45460, dtype: int32

In [185]:
md[md['vote_average'].notnull()]['vote_average'].mean()

5.618207215133889

In [186]:
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')

In [187]:
C = vote_averages.mean()
C

5.244896612406511

In [268]:
m = vote_counts.quantile(0.9)
m

160.0

In [354]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [355]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [356]:
qualified = md[(md['vote_count'] >= m)][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4555 entries, 0 to 45343
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         4555 non-null   object 
 1   year          4555 non-null   object 
 2   vote_count    4555 non-null   float64
 3   vote_average  4555 non-null   float64
 4   popularity    4555 non-null   object 
 5   genres        4555 non-null   object 
dtypes: float64(2), object(4)
memory usage: 249.1+ KB


In [357]:
qualified = md[(md['vote_count'] >= m)][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(4555, 6)

In [192]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [193]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [194]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [358]:
qualified.head()

,title,year,vote_count,vote_average,popularity,genres
0,Toy Story,1995,5415,7,21.9469,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,Jumanji,1995,2413,6,17.0155,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
4,Father of the Bride Part II,1995,173,5,8.38752,"[{'id': 35, 'name': 'Comedy'}]"
5,Heat,1995,1886,7,17.9249,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam..."
8,Sudden Death,1995,174,5,5.23158,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."


In [437]:
md['popularity'].map(type)

0        <class 'float'>
1        <class 'float'>
2        <class 'float'>
3        <class 'float'>
4        <class 'float'>
              ...       
45461      <class 'str'>
45462      <class 'str'>
45463      <class 'str'>
45464      <class 'str'>
45465      <class 'str'>
Name: popularity, Length: 45466, dtype: object

In [458]:
b= md[['popularity','id','original_language']]
b.head()

,popularity,id,original_language
0,21.9469,862,en
1,17.0155,8844,en
2,11.7129,15602,en
3,3.85949,31357,en
4,8.38752,11862,en


In [459]:
b.shape

(45466, 3)

In [460]:
b['popularity'].map(type)

0        <class 'float'>
1        <class 'float'>
2        <class 'float'>
3        <class 'float'>
4        <class 'float'>
              ...       
45461      <class 'str'>
45462      <class 'str'>
45463      <class 'str'>
45464      <class 'str'>
45465      <class 'str'>
Name: popularity, Length: 45466, dtype: object

In [463]:
b.drop(b[b['popularity'].map(type) != float].index,inplace=True)

In [464]:
b.shape

(32769, 3)

In [472]:
b.sort_values('popularity', ascending=False)

,popularity,id,original_language
30700,547.488,211672,en
24455,213.85,177572,en
26564,187.86,293660,en
26566,185.331,283995,en
14551,185.071,19995,en
...,...,...,...
19729,NaN,82663,en
19730,NaN,1997-08-20,104.0
29502,NaN,122662,ja
29503,NaN,2012-09-29,68.0


In [477]:
a= md.copy()

In [511]:
def is_float(s):
    popularity=[]
    for i in s:
        try:
            float(i)
            popularity.append(i)
        except:
            pass
    return popularity
        

In [512]:
is_float(a['popularity'])

[21.946943,
 17.015539,
 11.7129,
 3.859495,
 8.387519000000001,
 17.924927,
 6.677277,
 2.5611610000000002,
 5.23158,
 14.686035999999998,
 6.318445,
 5.430331,
 12.140733,
 5.092,
 7.284477000000001,
 10.137389,
 10.673167,
 9.026586,
 8.205447999999999,
 7.337905999999999,
 12.669608,
 10.701801,
 11.065939,
 12.133094,
 10.332025,
 1.8458990000000002,
 8.681325,
 2.228434,
 9.822422999999999,
 1.1009149999999999,
 9.481338000000001,
 12.297305,
 0.745542,
 14.404764000000002,
 1.4933610000000002,
 6.891317,
 0.114469,
 7.777735000000001,
 9.882383,
 0.894647,
 4.561387,
 9.87957,
 10.979269,
 10.870138,
 10.448481,
 4.485958,
 18.45743,
 13.280069000000001,
 3.302524,
 16.302466,
 0.595949,
 15.899134,
 1.361286,
 5.9827639999999995,
 4.093443,
 0.307075,
 8.416802,
 10.168436999999999,
 0.12078900000000001,
 10.673296,
 4.733526,
 7.137117,
 11.700115,
 2.174487,
 6.387469,
 2.49535,
 4.046208,
 2.518051,
 14.56965,
 15.339153,
 5.912223,
 3.3672760000000004,
 1.4794459999999998,


In [514]:
a['popularity']

0         21.9469
1         17.0155
2         11.7129
3         3.85949
4         8.38752
           ...   
45461    0.072051
45462    0.178241
45463    0.903007
45464    0.003503
45465    0.163015
Name: popularity, Length: 45466, dtype: object

In [491]:
a['popularity'].map(type)

0        <class 'float'>
1        <class 'float'>
2        <class 'float'>
3        <class 'float'>
4        <class 'float'>
              ...       
45461      <class 'str'>
45462      <class 'str'>
45463      <class 'str'>
45464      <class 'str'>
45465      <class 'str'>
Name: popularity, Length: 45466, dtype: object

In [196]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [197]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    df['year'] = pd.to_datetime(df['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
    
    #qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'vote_count', 'vote_average']]
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [198]:
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.565285
351,Forrest Gump,1994,8147,8,48.3072,7.971357
876,Vertigo,1958,1162,8,18.2082,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.8451,7.745154
1132,Cinema Paradiso,1988,834,8,14.177,7.744878
19901,Paperman,2012,734,8,7.19863,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.9943,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


## Content Based Recommender
- Movie Overviews and Targlines
- Movie Casts, Crew, Keywords and Genre

### Movie Overviews and Targlines

In [521]:
links_small = pd.read_csv('C:/Users/xiaoj/Desktop/CS/collection of data science-take home challenges/kaggle-Recommendation/links_small.csv')
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [522]:
links_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


In [524]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
links_small

KeyError: 'tmdbId'

In [202]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [518]:
md[md.index==19730]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [204]:
md = md.drop([19730, 29503, 35587])

In [205]:
md['id'] = md['id'].astype('int')
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

In [525]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [526]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [527]:
tfidf_matrix 

<9219x268124 sparse matrix of type '<class 'numpy.float64'>'
	with 548485 stored elements in Compressed Sparse Row format>

In [208]:
tfidf_matrix.shape

(9099, 268124)

With this matrix in hand, we can now compute a similarity score. There are several candidates for this; such as the euclidean, the Pearson and the cosine similarity scores.


We will be using the cosine similarity to calculate a numeric quantity that denotes the similarity between two movies. We use the cosine similarity score since it is independent of magnitude and is relatively easy and fast to calculate. 


Since we have used the TF-IDF vectorizer, calculating the dot product will directly give us the cosine similarity score. Therefore, we will use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.

In [209]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [210]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [211]:
def get_recommendations(title):
    # Get the index of the movie that matches the title
    idx = indices[title]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [212]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

### Movie Casts, Crew, Keywords and Genre

In [213]:
credits = pd.read_csv('C:/Users/xiaoj/Desktop/CS/collection of data science-take home challenges/kaggle-Recommendation/credits.csv')
keywords = pd.read_csv('C:/Users/xiaoj/Desktop/CS/collection of data science-take home challenges/kaggle-Recommendation/keywords.csv')

In [214]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [215]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [216]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [217]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [218]:
md.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  object 
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  object 
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  int32  
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45460 non-null  object 
 11  poster_path            45077 non-null  object 
 12  production_companies   45460 non-null  object 
 13  production_countries   45460 non-null  object 
 14  release_date           45376 non-null  object 
 15  re

In [219]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [222]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

In [223]:
smd.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [224]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9219 entries, 0 to 41669
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  9219 non-null   object 
 1   belongs_to_collection  1688 non-null   object 
 2   budget                 9219 non-null   object 
 3   genres                 9219 non-null   object 
 4   homepage               2001 non-null   object 
 5   id                     9219 non-null   int32  
 6   imdb_id                9219 non-null   object 
 7   original_language      9219 non-null   object 
 8   original_title         9219 non-null   object 
 9   overview               9207 non-null   object 
 10  popularity             9219 non-null   object 
 11  poster_path            9216 non-null   object 
 12  production_companies   9219 non-null   object 
 13  production_countries   9219 non-null   object 
 14  release_date           9219 non-null   object 
 15  rev

In [530]:
smd.head(1)

,level_0,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,...,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director,description
0,0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,...,7.7,5415.0,1995,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",13,106,"[johnlasseter, johnlasseter, johnlasseter]","Led by Woody, Andy's toys live happily in his ..."


In [529]:
#Right now, our data is present in the form of "stringified" lists , we need to convert it into a safe and usable structure
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    smd[feature] = smd[feature].apply(literal_eval)

ValueError: malformed node or string: ['tomhanks', 'timallen', 'donrickles']

In [225]:
# Right now, our data is present in the form of "stringified" lists , we need to convert it into a safe and usable structure
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [226]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [227]:
smd['director'] = smd['crew'].apply(get_director)

In [228]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [229]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [230]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9219 entries, 0 to 41669
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  9219 non-null   object 
 1   belongs_to_collection  1688 non-null   object 
 2   budget                 9219 non-null   object 
 3   genres                 9219 non-null   object 
 4   homepage               2001 non-null   object 
 5   id                     9219 non-null   int32  
 6   imdb_id                9219 non-null   object 
 7   original_language      9219 non-null   object 
 8   original_title         9219 non-null   object 
 9   overview               9207 non-null   object 
 10  popularity             9219 non-null   object 
 11  poster_path            9216 non-null   object 
 12  production_companies   9219 non-null   object 
 13  production_countries   9219 non-null   object 
 14  release_date           9219 non-null   object 
 15  rev

In [231]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

In [232]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [233]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [234]:
s = s[s > 1]

In [241]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [ ]:
stemmer = SnowballStemmer('english')

In [ ]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [ ]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

One important difference is that we use the CountVectorizer() instead of TF-IDF. This is because we do not want to down-weight the presence of an actor/director if he or she has acted or directed in relatively more movies. It doesn't make much intuitive sense.

In [265]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd)

In [261]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [262]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [266]:
get_recommendations('The Dark Knight').head(10)

IndexError: index 6981 is out of bounds for axis 0 with size 31

add a mechanism to remove bad movies and return movies which are popular and have had a good critical response.

I will take the top 25 movies based on similarity scores and calculate the vote of the 60th percentile movie. Then, using this as the value of  m , we will calculate the weighted rating of each movie using IMDB's formula

In [237]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

## Collaborative Filtering

The Surprise library that used extremely powerful algorithms like Singular Value Decomposition (SVD) to minimise RMSE (Root Mean Square Error) and give great recommendations.

In [239]:
ratings = pd.read_csv('C:/Users/xiaoj/Desktop/CS/collection of data science-take home challenges\kaggle-Recommendation/ratings_small.csv')
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [244]:
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE','MAE'], cv=5)

{'test_rmse': array([0.89229498, 0.89998225, 0.89894022, 0.89314851, 0.90032888]),
 'test_mae': array([0.68670024, 0.6918187 , 0.69292781, 0.68916344, 0.69193228]),
 'fit_time': (6.417524814605713,
  8.142713069915771,
  8.30793309211731,
  10.049753665924072,
  7.663072109222412),
 'test_time': (0.16963744163513184,
  0.19544410705566406,
  0.1882307529449463,
  0.18550419807434082,
  0.19148874282836914)}

In [245]:
trainset = data.build_full_trainset()
svd.train(trainset)

AttributeError: 'SVD' object has no attribute 'train'

In [246]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [247]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.8394889875203244, details={'was_impossible': False})

For movie with ID 302, we get an estimated prediction of 2.686. One startling feature of this recommender system is that it doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.

## Hybrid Recommender

In [248]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [249]:
id_map = pd.read_csv('C:/Users/xiaoj/Desktop/CS/collection of data science-take home challenges\kaggle-Recommendation/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [250]:
indices_map = id_map.set_index('id')

In [251]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [252]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
1180,"The Good, the Bad and the Ugly",2371.0,8.1,1966,429,3.361924
7016,Nausicaä of the Valley of the Wind,808.0,7.7,1984,81,3.175385
6367,Duel at Diablo,22.0,6.3,1966,1403,3.002948
4101,Double Impact,219.0,5.3,1991,9594,2.963343
16462,True Grit,1701.0,7.2,2010,44264,2.924585
3457,Caddyshack,370.0,6.7,1980,11977,2.905869
15219,Steam of Life,11.0,6.9,2010,52903,2.824212
9791,The Nomi Song,5.0,6.7,2004,41569,2.806237
3076,Brenda Starr,7.0,5.1,1989,47070,2.801467
6603,Fire,17.0,6.0,1996,513,2.801467
